# Лекция 1. Арифметика чисел с плавающей точкой.

## План на сегодня

- Арифметика чисел с фиксированной и плавающей точкой
- Форматы представления действительных чисел в памяти компьютера
- Почему способы хранения чисел в компьютере важны для обучения нейросетей

## Представление чисел 

- Действительные числа обозначают различные величины: вероятности, массы, скорости, длины...

- Важно знать, как они представляются в компьютере, который оперирует только битами.

## Представление с фиксированной точкой

- Наиболее очевидный формат представления чисел – это формат чисел с **фиксированной точкой**, также известный как **Qm.n** формат

- Число **Qm.n** лежит в отрезке $[-(2^m), 2^m - 2^{-n}]$ и его точность $2^{-n}$.

- Общий размер памяти для хранения такого числа $m + n + 1$ бит.

- Диапазон чисел, представимых таким образом, ограничен.

## Числа с плавающей точкой (floating point numbers)

Числа в памяти компьютера обычно представляются в виде **чисел с плавающей точкой.**

Число с плавающей точкой представляется в виде

$$ \textrm{number} = \textrm{significand} \times \textrm{base}^{\textrm{exponent}},$$

где *significand* – целое число (aka мантисса), *base* – натуральное число (основание) и *exponent* – целое число (может быть отрицательным), например

$$ 1.2 = 12 \cdot 10^{-1}.$$

## Формат фиксированной точки vs формат плавающей точки

**Q**: какие достоинства и недостатки у рассмотренных форматов представления действительных чисел?

**A**: они подходят для большинства случаев.

- Однако, числа с фиксированной точкой представляют числа из фиксированного интервала и ограничивают **абсолютную** точность.

- Числа с плавающей точкой представлют числа с **относительной** точностью и удобны для случаев, когда числа, которые участвуют в вычислениях, имеют различный порядок (например $10^{-1}$ и $10^{5}$).

- На практике, если скорость не критически важна, стоит использовать float32 или float64.

## IEEE 754
В современных компьютерах представление чисел в виде чисел с плавающей точкой регулируется стандартом [IEEE 754](https://en.wikipedia.org/wiki/IEEE_floating_point), который был опубликован в **1985 г.** До этого компьютеры обрабатывали числа с плавающей точкой по-разному!

IEEE 754 содержит следующие элементы:
- Представление чисел с плавающей точкой (как было описано выше), $(-1)^s \times c \times b^q$.
- Две бесконечности $+\infty$ and $-\infty$
- Два типа **NaN**: "тихий" NaN (**qNaN**) and сигнализирующий NaN (**sNaN**) 
    - qNaN не бросает исключение на уровне блока, производящего операции с плавающей точкой, (floating point unit – FPU), до того как вы проверите результат вычислений
    - значение sNaN бросает исключение из FPU, если вы используете это значение в вычислениях. Этот тип NaN может быть полезен для инициализиции
    - C++11 имеет [стандартный интерфейс](https://en.cppreference.com/w/cpp/numeric/math/nan) для создания разных типов NaN
- Правила **округления**
- Правила для операций типа $\frac{0}{0}, \frac{1}{-0}, \ldots$

Возможные значения определяются через
- основание $b$
- точность $p$ - число цифр в записи
- максимально возможное значение $e_{\max}$

и имеет следующие ограничения

- $ 0 \leq c \leq b^p - 1$
- $1 - e_{\max} \leq q + p - 1 \leq e_{\max}$ 

## Два наиболее используемых формата: single & double

Наиболее часто используются следующие форматы: **binary32** и **binary64** (также известные как **single** и **double**).
В последнее время популярность набирают вычисления с половинной точностью **binary16**.

| Формальное название | Другое название | Основание | Число цифр в записи | Emin | Emax |
|------|----------|----------|-------|------|------|
|binary16| half precision  | 2 | 11 | -14 | + 15 |  
|binary32| single precision | 2 | 24 | -126 | + 127 |  
|binary64| double precision | 2|  53|  -1022| +1023|

Как выглядит формат двойной точности ```double```

<img src="./double64.png">


## Примеры

- Для числа +0
    - *sign* - 0
    - *exponent* - 00000000000
    - *fraction* - все нули
- Для числа -0
    - *sign* - 1
    - *exponent* - 00000000000
    - *fraction* - все нули
- Для +infinity
    - *sign* - 0
    - *exponent* - 11111111111
    - *fraction* - все нули

**Q**: Как будет выглядеть -infinity и NaN ?

## Точность и размер памяти 

**Относительная точность** для различных форматов

- половинная точность или float16: $10^{-3} - 10^{-4}$,
- одинарная точность или float32: $10^{-7}-10^{-8}$,
- двойная точность или float64: $10^{-14}-10^{-16}$.

<font color='red'> Crucial note 1: </font> **float16** занимает **2 байта**, **float32** занимает **4 байта**, **float64** занимает **8 байт**

<font color='red'> Crucial note 2: </font> Обычно в "железе" поддерживается одинарная и двойная точность. Для обучения нейросетей становится популярным использовать половинную точность или даже меньше... Подробности [тут](https://arxiv.org/pdf/1905.12334.pdf)

## Альтернатива стандарту IEEE 754

Недостатки IEEE 754:
- переполнения до бесконечности или нуля
- много разных NaN
- невидимые ошибки округления (об этом ниже)
- точность либо очень хорошая, либо очень плохая
- субнормальные числа – числа между 0 и минимально возможным представимым числом, то есть мантисса субнормального числа начинается с ведущего 0

Концепция posits может заменить числа с плавающей точкой, см [статью](http://www.johngustafson.net/pdfs/BeatingFloatingPoint.pdf)

<img width=600 src="./posit.png">

- выражают числа с некоторой точностью, но указывают предел изменения
- отсутствуют переполнения до бесконечности или нуля
- пример хранения числа

<img width=600 src="./posit_example.png">

### Пример 1: потеря точности при делении

In [8]:
import numpy as np
import random
#c = random.random()
#print(c)
c = np.float32(0.925924589693)
print(c)
a = np.float32(8.9)
b = np.float32(c / a)
print('{0:10.16f}'.format(b))
print(a * b - c)

0.9259246
0.1040364727377892
-5.9604645e-08


### Пример 2: потеря точности при извлечении корня

In [17]:
#a = np.array(1.585858585887575775757575e-5, dtype=np.float)
a = np.float32(5.0)
b = np.sqrt(a)
print('{0:10.16f}'.format((b ** 2 - a) / a))
print((b ** 2 - a))

0.0000000293644121
1.468220602873771e-07


### Пример 3: потеря точности при вычислении экспоненты

In [23]:
a = np.array(2.2882727271099, dtype=np.float16)
b = np.log(a)
print(np.exp(b) - a)

0.0


## Выводы

- Для некоторых чисел обратные функции дают неточный ответ
- Относительная точность должна сохраняться в соответствии со стандартом IEEE
- Это требование не выполняется на многих современных GPU
- Подробный анализ выполнимости стандарта IEEE 754 на GPU от NVIDIA можно найти [здесь](https://docs.nvidia.com/cuda/floating-point/index.html#considerations-for-heterogeneous-world) 

## Потеря значимых цифр

- Многие операции приводят к потере значимых цифр в результате, этот эффект называется [loss of significance](https://en.wikipedia.org/wiki/Loss_of_significance)
- Например, при вычитании двух близких больших чисел результат будет иметь меньше правильных цифр, чем исходные цифры
- Это связано с алгоритмами и их свойствами (прямой/обратной устойчивости), которые мы обсудим далее

In [26]:
a = np.float64(123456789999999.11)
print(a)
b = np.float64(123456789999998.1)
print(b)
c = a - b
print('{0:10.17f}'.format(c))

123456789999999.11
123456789999998.1
1.01562500000000000


## Алгоритм суммирования

Однако ошибка округления зависит от алгоритма вычисления.

- Рассмотрим простейшую задачу: дано $n$ чисел с плавающей точкой $x_1, \ldots, x_n$  

- Необходимо вычислить их сумму

$$ S = \sum_{i=1}^n x_i = x_1 + \ldots + x_n.$$

- Простейший алгоритм: складывать числа $x_1, \ldots, x_n$ одно за другим

- Какова ошибка такого алгоритма при работе в неточной арифметике?

## Наивный алгоритм

Сложим числа одно за другим: 

$$y_1 = x_1, \quad y_2 = y_1 + x_2, \quad y_3 = y_2 + x_3, \ldots.$$

- В **худшем случае** ошибка пропорциональна $\mathcal{O}(n)$, в то время как **средне-квадратичная** ошибка $\mathcal{O}(\sqrt{n})$.

- **Алгоритм Кахана** (Kahan algorithm) даёт ошибку в худшем случае $\mathcal{O}(1)$ (то есть она не зависит от $n$).  

## Алгоритм Кахана (Kahan summation)

Следующий алгоритм даёт ошибку $2 \varepsilon + \mathcal{O}(n \varepsilon^2)$, где $\varepsilon$ – машинная точность.
```python
s = 0
c = 0
for i in range(len(x)):
    y = x[i] - c
    t = s + y
    c = (t - s) - y
    s = t
```

- Пример использования такого метода в библиотеке глубокого обучения PyTorch - https://github.com/pytorch/torchdistx/pull/52

In [32]:
import math

import jax.numpy as jnp
import numpy as np
import jax
from numba import jit as numba_jit

n = 10 ** 6
sm = 1e-10
x = jnp.ones(n, dtype=jnp.float32) * sm
x = x.at[0].set(1.)
true_sum = 1.0 + (n - 1)*sm
approx_sum = jnp.sum(x)
math_fsum = math.fsum(x)


@jax.jit
def dumb_sum(x):
    s = jnp.float32(0.0)
    def b_fun(i, val):
        return val + x[i] 
    s = jax.lax.fori_loop(0, len(x), b_fun, s)
    return s


@numba_jit(nopython=True)
def kahan_sum_numba(x):
    s = np.float32(0.0)
    c = np.float32(0.0)
    for i in range(len(x)):
        y = x[i] - c
        t = s + y
        c = (t - s) - y
        s = t
    return s

@jax.jit
def kahan_sum_jax(x):
    s = jnp.float32(0.0)
    c = jnp.float32(0.0)
    def b_fun2(i, val):
        s, c = val
        y = x[i] - c
        t = s + y
        c = (t - s) - y
        s = t
        return s, c
    s, c = jax.lax.fori_loop(0, len(x), b_fun2, (s, c))
    return s

k_sum_numba = kahan_sum_numba(np.array(x))
k_sum_jax = kahan_sum_jax(x)
d_sum = dumb_sum(x)
print('Error in np sum: {0:3.1e}'.format(approx_sum - true_sum))
print('Error in Kahan sum Numba: {0:3.1e}'.format(k_sum_numba - true_sum))
print('Error in Kahan sum JAX: {0:3.1e}'.format(k_sum_jax - true_sum))
print('Error in dumb sum: {0:3.1e}'.format(d_sum - true_sum))
print('Error in math fsum: {0:3.1e}'.format(math_fsum - true_sum))

Error in np sum: -2.4e-07
Error in Kahan sum Numba: 1.7e-08
Error in Kahan sum JAX: 0.0e+00
Error in dumb sum: -1.0e-04
Error in math fsum: 1.3e-12


## Ещё один пример

In [33]:
import numpy as np
test_list = [1, 1e20, 1, -1e20]
print(math.fsum(test_list))
print(np.sum(test_list))
print(1 + 1e20 + 1 - 1e20)

2.0
0.0
0.0


## Выводы по операциям с числами с плавающей точкой

- Существуют различные форматы и подходы к хранению чисел в памяти
- Операции с числами с плавающей точкой могут приводить к неточным ответам из-за ошибок округления - требуется осторожность!
- Для многих стандартных алгоритмов вопросы устойчивости хорошо изучены и проблемы, связанные с этим, легко обнаруживаются.

## Кратко про нейросети

- Элементарные функции могут быть параметризованы, например $f(x) = Wx + b$ – линейная функция с параметрами $W$ и $b$
- Нейросеть – это суперпозиция параметрических функций
- Параметры элементарных функций, из которых состоит нейросеть, задаются с помощью чисел с плавающей точкой
- Формат хранения параметров напрямую влияет на то, насколько точно и устойчиво будет вычисляться итоговый результат

### Форматы представления чисел и нейронные сети

- Для быстрой и энергоэффективной работы с нейросетями разработаны специальные форматы хранения чисел (подробности далее)
- Они помогают ускорить получение обученных моделей с некоторой потерей точности промежуточных вычислений

## Влияние формата представления чисел на обучение нейросетей

- Веса в слоях (полносвязном, свёрточном, действие функций активации) могут храниться с различной точностью
- Это важно для повышении энергоэффективности устройств, на которых запускается обученная нейросеть
- Проект [DeepFloat](https://github.com/facebookresearch/deepfloat) от Facebook показывает, как переизобрести операции с плавающей точкой, чтобы они были эффективны для обучения нейросетей, подробности см. в [статье](https://arxiv.org/pdf/1811.01721.pdf)
- Влияние формата представления чисел на значения градиентов активаций

<img width=500, src="./grad_norm_fp16.png">

- И на кривые обучения

<img width=500, src="./train_val_curves.png">

Графики взяты из [этой статьи](https://arxiv.org/pdf/1710.03740.pdf%EF%BC%89%E3%80%82)

- Проект [bitsandbytes](https://github.com/TimDettmers/bitsandbytes) предлагает ядра для CUDA, где реализованы базовые операции с малобитным представлением чисел

## bfloat16 (Brain Floating Point)

- Этот формат требует 16 битов
    - 1 бит для знака
    - 8 битов для экспоненты
    - 7 bits для мантиссы
    <img src="./bfloat16.png">
- Усечённая одинарная точность из стандарта IEEE
- Какая разница между float32 и float16?
- Этот формат используется в Intel FPGA, Google TPU, Xeon CPUs и других платформах

## Tensor Float от Nvidia ([блог об этом формате](https://blogs.nvidia.com/blog/2020/05/14/tensorfloat-32-precision-format/))

- Сравнение с другими форматами

<img src="./tensor_float_cf.png">

- Результаты

<img src="./TF32-BERT.png">

- PyTorch и Tensorflow, поддерживающие этот формат, доступны в [Nvidia NGC](https://ngc.nvidia.com/catalog/all)

## Смешанная точность ([документация от Nvidia](https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html))

- Основная идея:
    - Поддерживать копии весов в одинарной точности
    - Тогда на каждой итерации
        - Сделать копию весов в половинной точности
        - Сделать проход вперёд с весами в половинной точности
        - Умножить значение функции ошибки на множитель $S$
        - Вычислить градиент в половинной точности
        - Умножить градиенты весов на $1/S$
        - Обновить веса 
    - Множитель $S$ это гиперпараметр
    - Постоянный: такое значение, что его произведение на максимальное абсолютное значение градиента меньше чем 65504 (максимальное число, представимое в половинной точности).
    - Динамическое обновление, основанное на статистиках текущих градиентов
- Сравнение производительности
<img src="./mixed_precision_res.png" width=500>

- Существуют расширения для автоматического включения этой опции, больше подробностей [тут](https://developer.nvidia.com/automatic-mixed-precision)

## Резюме

- Числа с плавающей точкой – основа для проведения вычислений
- Использование нейросетей поставило новые вопросы и предъявило новые требования к числам и операциям с ними на уровне железа
- Помимо использования фиксированнго формата применяют смешанную точность